<!-- Banner Image -->
<center>
    <img src="https://developer-blogs.nvidia.com/wp-content/uploads/2024/07/rag-representation.jpg" width="75%">
</center>

<!-- Links -->
<center>
  <a href="https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/workbench/" style="color: #76B900;">NVIDIA AI Workbench</a> •
  <a href="https://docs.nvidia.com/ai-workbench/" style="color: #76B900;">User Documentation</a> •
  <a href="https://docs.nvidia.com/ai-workbench/user-guide/latest/quickstart/example-projects.html" style="color: #76B900;">Example Projects Catalog</a> •
  <a href="https://forums.developer.nvidia.com/t/support-workbench-example-project-llama-3-finetune/303411" style="color: #76B900;"> Problem? Submit a ticket here! </a>
</center>

# Finetune and deploy the Llama3-8b model using SFT and VLLM 

Welcome!

In this notebook, we're going to walk through the flow of using supervised finetuning (SFT) on the Llama3-8B model from scratch using the base model and then deploying it using VLLM. Ensure you have requested and have been approved access to this model via [HuggingFace](https://huggingface.co/meta-llama/Meta-Llama-3-8B). 

Llama-3 was has an 8k context length which is pretty small compared to some of the newer models that have been released and is was pretrained with 15 trillion tokens on a 24k GPU cluster. Luckily for finetuning, we only need a fraction of that compute power.

Note that we will be using the base model in this notebook and not the instruct model. Additionally, we will be running through a full finetune with no quantization. This notebook was originally built for 2 A100-80GB GPUs, but the default hyperparameters have since been adjusted to run on 1x A100-80GB. If you're looking for a lighter Llama3-finetune, checkout out the other Llama3 finetuning notebook which uses Direct Preference Optimization.

#### Help us make this tutorial better! Please provide feedback on the [NVIDIA Developer Forum](https://forums.developer.nvidia.com/c/ai-data-science/nvidia-ai-workbench/671).

A note about running Jupyter Notebooks: Press Shift + Enter to run a cell. A * in the left-hand cell box means the cell is running. A number means it has completed. If your Notebook is acting weird, you can interrupt a too-long process by interrupting the kernel (Kernel tab -> Interrupt Kernel) or even restarting the kernel (Kernel tab -> Restart Kernel). Note restarting the kernel will require you to run everything from the beginning.

## Table of Contents
1. Import libraries
2. Download model
3. Fintuning flow
4. Deploy as an OpenAI compatible endpoint

## 1. Imports libraries

In [1]:
# import sys, importlib.util

# print("Python:", sys.executable)

# spec = importlib.util.find_spec("rich")
# if spec is None:
#     print("rich: NOT installed in this kernel")
# else:
#     import rich
#     print("rich: INSTALLED, version =", rich.__version__)


In [2]:
# import sys
# !{sys.executable} -m pip install -U rich

# from trl import SFTTrainer


In [3]:
# import sys
# print(sys.executable)
# !{sys.executable} -m pip show trl rich


In [4]:
# SPDX-FileCopyrightText: Copyright (c) 2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: Apache-2.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:215: UserWarning: 
NVIDIA RTX PRO 6000 Blackwell Workstation Edition with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_52 sm_60 sm_61 sm_70 sm_72 sm_75 sm_80 sm_86 sm_87 sm_90 compute_90.
If you want to use the NVIDIA RTX PRO 6000 Blackwell Workstation Edition GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


## 2. Load in Llama 3 and our dataset

Because we are using the base model, there is not an exact prompt template we have to follow. The dataset we are using follows LLama3's template format so it should be fine for downstream tasks that use the Llama3 chat format. If you're bringing your own data, you can format it however you want as long as you use the same formatting downstream. 

Here's the official [Llama3 chat template](https://huggingface.co/blog/llama3#how-to-prompt-llama-3)

In [5]:
base_model_id = "meta-llama/Meta-Llama-3-8B"
dataset_name = "scooterman/guanaco-llama3-1k"
new_model = "/project/models/NV-llama3-8b-SFT"

In [6]:
from datasets import load_dataset

dataset = load_dataset(dataset_name, split="train")

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model = AutoModelForCausalLM.from_pretrained(base_model_id, 
                                             token=os.environ["HF_KEY"], 
                                             cache_dir="/project/models",
                                             device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    token=os.environ["HF_KEY"], 
    add_eos_token=True,
    add_bos_token=True, 
)
tokenizer.pad_token = tokenizer.eos_token

/home/workbench/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B.
403 Client Error. (Request ID: Root=1-6958a9c5-4e5c64197c5e5b172ca0eb1f;fd33052a-cd69-4708-9f39-fbbd1b0bcd52)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Meta-Llama-3-8B to ask for access.

## 3. Set our Training Arguments

A lot of tutorials simply paste a list of arguments leaving it up to the reader to figure out what each argument does. Below, annotations have been added to explain what each argument does.

In [ ]:
# Output directory where the results and checkpoint are stored
output_dir = "./results"

# Number of training epochs - how many times does the model see the whole dataset
num_train_epochs = 1 #Increase this for a larger finetune

# Enable fp16/bf16 training. This is the type of each weight. Since we are on an A100
# we can set bf16 to true because it can handle that type of computation
bf16 = True

# Batch size is the number of training examples used to train a single forward and backward pass. 
per_device_train_batch_size = 1

# Gradients are accumulated over multiple mini-batches before updating the model weights. 
# This allows for effectively training with a larger batch size on hardware with limited memory
gradient_accumulation_steps = 8

# memory optimization technique that reduces RAM usage during training by intermittently storing 
# intermediate activations instead of retaining them throughout the entire forward pass, trading 
# computational time for lower memory consumption.
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Number of training steps (overrides num_train_epochs)
max_steps = 500

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 100

# Log every X updates steps
logging_steps = 5

## (Optional) Run the training using WandB for logging

Weights and Biases is industry standard for monitoring and evaluating your training job. If you have an account and API key, you can monitor this run.

In [ ]:
### Uncomment to use Weights and Biases ###

# import wandb

# wandb.login()

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    gradient_checkpointing=gradient_checkpointing,
    report_to="none" # can replace with "wandb"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

## 4. Run the model for inference!

To deploy this model for extremely quick inference, we use VLLM and host an OpenAI compatible endpoint. You might have to **restart the kernel** to flush the GPU memory and then just run the cells below. 

First, uncomment and install the vLLM pip package. You may need to restart the kernel for the package to take effect. Then, run the command to start the API server. Once running, you can open a new tab in Jupyterlab, select Terminal, and run the curl command to send a request to the server. 

**Note:** We install the latest version of vLLM in the following cell. This may upgrade your transformers package version, which can cause issues if you re-run the notebook from the beginning. If you would like to re-run the entire notebook for another finetuning flow, restart the project environment from inside AI Workbench to get a fresh environment to work in.

In [ ]:
# Uncomment to install the latest version of vLLM. 
# !pip install vllm

In [ ]:
!python -O -u -m vllm.entrypoints.openai.api_server \
    --host=0.0.0.0 \
    --port=8000 \
    --model=/project/models/NV-llama3-8b-SFT \
    --tokenizer=meta-llama/Meta-Llama-3-8B \
    --tensor-parallel-size=1 # set to number of GPUs

# Open up a terminal and run
# curl http://localhost:8000/v1/completions \
#    -H "Content-Type: application/json" \
#    -d '{
#        "model": "/project/models/NV-llama3-8b-SFT",
#        "prompt": "What is San Francisco",
#        "max_tokens": 30,
#        "temperature": 0
#    }'